In [1]:
# No-Free-Miracle Test Bench (Self-contained, single cell)

import math, random, itertools, sys, statistics
from dataclasses import dataclass
from typing import List, Dict, Tuple, Callable, Optional
import numpy as np

try:
    import mpmath as mp
except ImportError:
    import sys, subprocess; subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'mpmath'])
    import mpmath as mp

mp.mp.dps = 80  # working precision (digits)

# ------------------------------- Fixed targets (predeclared) -------------------------------
def phi():
    return (mp.mpf(1) + mp.sqrt(5)) / 2

# CODATA α^{-1} values differ at ~1e-9; include both so we don't cherry-pick later.
ALPHA_INV_CODATA_2018 = mp.mpf("137.035999084")
ALPHA_INV_CODATA_2022 = mp.mpf("137.035999206")

targets: Dict[str, mp.mpf] = {
    "pi": mp.pi,
    "e": mp.e,
    "phi": phi(),
    "sqrt2": mp.sqrt(2),
    "sqrt3": mp.sqrt(3),
    "zeta3": mp.zeta(3),                  # Apery's constant
    "CatalanG": mp.catalan,
    "Feigenbaum_delta": mp.mpf("4.66920160910299067185320382"),
    "alpha_inv_2018": ALPHA_INV_CODATA_2018,
    "alpha_inv_2022": ALPHA_INV_CODATA_2022,
    "alpha_2018": 1/ALPHA_INV_CODATA_2018,
    "alpha_2022": 1/ALPHA_INV_CODATA_2022,
    "alpha_inv_div_100_2018": ALPHA_INV_CODATA_2018/100,
    "alpha_inv_div_100_2022": ALPHA_INV_CODATA_2022/100,
}

# ------------------------------- Generators (no knobs) -------------------------------------
def fibonacci_word_bits(n: int) -> List[int]:
    """Fibonacci word via morphism 0->01, 1->0 (start '0')."""
    s = "0"
    while len(s) < n:
        s = s.replace("0", "x")
        s = s.replace("1", "0")
        s = s.replace("x", "01")
    s = s[:n]
    return [1 if c == '1' else 0 for c in s]

def thue_morse_bits(n: int) -> List[int]:
    """Thue–Morse: parity of ones in binary index."""
    out = []
    for k in range(n):
        out.append((bin(k).count("1") & 1))
    return out

def random_balanced_bits(n: int, rng: random.Random) -> List[int]:
    return [1 if rng.random() < 0.5 else 0 for _ in range(n)]

# ---------------------------- Stage 1 maps (fixed grammar) --------------------------------
def seq_to_binary_fraction(bits: List[int]) -> mp.mpf:
    """Interpret bits as 0.b1 b2 ... (base-2)."""
    x = mp.mpf(0); half = mp.mpf(0.5); w = half
    for b in bits:
        if b == 1: x += w
        w *= half
    return x

def sign_map(bits: List[int]) -> List[int]:
    return [1 if b == 1 else -1 for b in bits]

def seq_dirichlet_all(bits: List[int], s: int) -> mp.mpf:
    """Σ_{k=1..N} sgn(b_k) / k^s"""
    sgn = sign_map(bits); N = len(bits); total = mp.mpf(0)
    for k in range(1, N+1):
        total += sgn[k-1] / (mp.mpf(k) ** s)
    return total

def sieve_primes(n: int) -> List[int]:
    if n < 2: return []
    A = [True]*(n+1); A[0]=A[1]=False; p=2
    while p*p <= n:
        if A[p]:
            for j in range(p*p, n+1, p): A[j]=False
        p += 1
    return [i for i,flag in enumerate(A) if flag]

def seq_dirichlet_primes(bits: List[int], s: int) -> mp.mpf:
    """Σ_{p prime ≤ N} sgn(b_p) / p^s  (bit at index p-1)."""
    sgn = sign_map(bits); N = len(bits); primes = sieve_primes(N); total = mp.mpf(0)
    for p in primes:
        total += sgn[p-1] / (mp.mpf(p) ** s)
    return total

def seq_golden_damped(bits: List[int], s: int = 1) -> mp.mpf:
    """Σ_{k=1..N} sgn(b_k) * φ^{-k} / k^s  (φ fixed; no fit)."""
    sgn = sign_map(bits); N = len(bits); ph = phi(); ph_inv = 1/ph
    total = mp.mpf(0); pow_phi = ph_inv
    for k in range(1, N+1):
        total += sgn[k-1] * pow_phi / (mp.mpf(k) ** s)
        pow_phi *= ph_inv
    return total

STAGE1_FAMILIES: Dict[str, Tuple[Callable, Dict]] = {
    "BinaryFraction":   (seq_to_binary_fraction, {}),
    "DirichletAll_s1":  (seq_dirichlet_all,   {"s":1}),
    "DirichletAll_s2":  (seq_dirichlet_all,   {"s":2}),
    "DirichletPrime_s1":(seq_dirichlet_primes,{"s":1}),
    "GoldenDamped_s1":  (seq_golden_damped,   {"s":1}),
}

# ---------------------------- Stage 2 transforms (fixed set) -------------------------------
def id_transform(x: mp.mpf) -> Optional[mp.mpf]: return x
def reciprocal(x: mp.mpf) -> Optional[mp.mpf]: return None if x == 0 else 1/x
def reciprocal_div_100(x: mp.mpf) -> Optional[mp.mpf]: return None if x == 0 else 1/(100*x)
def times_729(x: mp.mpf) -> Optional[mp.mpf]: return mp.mpf(729) * x
def reciprocal_times_729(x: mp.mpf) -> Optional[mp.mpf]: return None if x == 0 else mp.mpf(729)/x

STAGE2_TRANSFORMS: Dict[str, Callable[[mp.mpf], Optional[mp.mpf]]] = {
    "id": id_transform,
    "reciprocal": reciprocal,
    "reciprocal_div_100": reciprocal_div_100,
    "times_729": times_729,
    "reciprocal_times_729": reciprocal_times_729,
}

SEQUENCES: Dict[str, Callable[[int], List[int]]] = {
    "FibonacciWord": fibonacci_word_bits,
    "ThueMorse":     thue_morse_bits,
}

# Trial sizes (truncation lengths); these count toward look-elsewhere penalty
N_LIST = [256, 512, 1024]

# --------------------------- Digit comparison (scientific form) ----------------------------
def sci_digits(x: mp.mpf, M: int) -> Tuple[int, str]:
    """Return (exponent k, first M mantissa digits of |x| in scientific notation)."""
    if x == 0: return (0, "0"*M)
    x = mp.fabs(x)
    k = int(mp.floor(mp.log10(x)))
    d = x / (mp.mpf(10) ** k)  # 1 <= d < 10
    s = mp.nstr(d, n=M+1, strip_zeros=False)  # e.g. '1.2345'
    if "e" in s or "E" in s:
        s = mp.nstr(d, n=M+1, strip_zeros=False)
    if "." in s:
        a, b = s.split("."); digits = a + b
    else:
        digits = s
    if len(digits) < M: digits += "0"*(M-len(digits))
    return k, digits[:M]

def shared_prefix_len(a: str, b: str) -> int:
    m = min(len(a), len(b)); c = 0
    for i in range(m):
        if a[i] == b[i]: c += 1
        else: break
    return c

def digits_match(x: mp.mpf, y: mp.mpf, M: int, offset: int = 0) -> Tuple[int, int]:
    """(D, exp_equal). Compare mantissa digits starting at offset; exponents must match."""
    kx, sx = sci_digits(x, M+offset)
    ky, sy = sci_digits(y, M+offset)
    if kx != ky: return 0, 0
    return shared_prefix_len(sx[offset:], sy[offset:]), 1

# ------------------------------ Search space (for T accounting) ----------------------------
@dataclass(frozen=True)
class PipelineSpec:
    seq_name: str
    N: int
    stage1_name: str
    stage2_name: str

def all_pipelines() -> List[PipelineSpec]:
    for seq_name in SEQUENCES.keys():
        for N in N_LIST:
            for s1 in STAGE1_FAMILIES.keys():
                for s2 in STAGE2_TRANSFORMS.keys():
                    yield PipelineSpec(seq_name, N, s1, s2)

PIPELINES = list(all_pipelines())
T_TOTAL = len(PIPELINES)            # number of *independent trials*
LOG2_T = math.log2(T_TOTAL)

# ------------------------------- Evaluation configuration ---------------------------------
M_TOTAL = 50     # total digits available for comparison
M_TRAIN = 14     # digits used to select the best pipeline (per target)
M_TEST  = 14     # held-out digits after the first M_TRAIN

assert M_TRAIN + M_TEST <= M_TOTAL

# --------------------------------- Core evaluation ----------------------------------------
def run_pipeline(spec: PipelineSpec) -> Optional[mp.mpf]:
    bits = SEQUENCES[spec.seq_name](spec.N)
    s1_fn, s1_kwargs = STAGE1_FAMILIES[spec.stage1_name]
    x = s1_fn(bits, **s1_kwargs) if s1_kwargs else s1_fn(bits)
    x = mp.fabs(x)  # predeclared: compare absolute values only
    return STAGE2_TRANSFORMS[spec.stage2_name](x)

def evaluate_against_targets():
    values: Dict[PipelineSpec, Optional[mp.mpf]] = {}
    for spec in PIPELINES:
        try:
            values[spec] = run_pipeline(spec)
        except Exception:
            values[spec] = None

    results = {}
    for tname, tval in targets.items():
        best = None
        for spec, out in values.items():
            if out is None: continue
            D_train, exp_eq = digits_match(out, tval, M_TRAIN, offset=0)
            if exp_eq and (best is None or D_train > best["D_train"]):
                # compute held-out on next M_TEST digits
                _, sx = sci_digits(out, M_TRAIN+M_TEST)
                _, sy = sci_digits(tval, M_TRAIN+M_TEST)
                kx, _ = sci_digits(out, 1); ky, _ = sci_digits(tval, 1)
                D_test = shared_prefix_len(sx[M_TRAIN:M_TRAIN+M_TEST], sy[M_TRAIN:M_TRAIN+M_TEST]) if kx==ky else 0
                best = {"spec": spec, "value": out, "D_train": D_train, "D_test": D_test}
        if best is not None:
            results[tname] = {**best, "surplus_bits_train": 3.322*best["D_train"] - LOG2_T, "log2_T": LOG2_T}
        else:
            results[tname] = None
    return results

# ----------------------------- Randomized baselines (robustness) --------------------------
def evaluate_random_baseline(num_runs: int = 20, seed: int = 123):
    rng = random.Random(seed)
    max_D_by_target = {t: [] for t in targets.keys()}
    for run in range(num_runs):
        values = {}
        cache_bits = {}
        for spec in PIPELINES:
            key = (spec.seq_name, spec.N, run)
            if key not in cache_bits:
                cache_bits[key] = random_balanced_bits(spec.N, rng)
            bits = cache_bits[key]
            s1_fn, s1_kwargs = STAGE1_FAMILIES[spec.stage1_name]
            try:
                x = s1_fn(bits, **s1_kwargs) if s1_kwargs else s1_fn(bits)
                x = mp.fabs(x)
                x = STAGE2_TRANSFORMS[spec.stage2_name](x)
            except Exception:
                x = None
            values[spec] = x

        for tname, tval in targets.items():
            best_D = 0
            for spec, out in values.items():
                if out is None: continue
                D_train, exp_eq = digits_match(out, tval, M_TRAIN, offset=0)
                if exp_eq and D_train > best_D: best_D = D_train
            max_D_by_target[tname].append(best_D)

    summaries = {}
    for tname, arr in max_D_by_target.items():
        summaries[tname] = {
            "mean": statistics.mean(arr),
            "stdev": statistics.pstdev(arr),
            "max": max(arr),
            "hist": arr,
        }
    return summaries

# ----------------------------- Local robustness: bit flips --------------------------------
def flip_fraction(bits: List[int], frac: float, rng: random.Random) -> List[int]:
    n = len(bits); k = max(1, int(round(frac * n)))
    idx = list(range(n)); rng.shuffle(idx); flip = set(idx[:k])
    return [(1-b) if i in flip else b for i,b in enumerate(bits)]

def robustness_check(best_spec: PipelineSpec, perturb_frac: float = 0.01, trials: int = 20, seed: int = 321):
    rng = random.Random(seed)
    base_bits = SEQUENCES[best_spec.seq_name](best_spec.N)
    s1_fn, s1_kwargs = STAGE1_FAMILIES[best_spec.stage1_name]
    def value_from_bits(bits):
        x = s1_fn(bits, **s1_kwargs) if s1_kwargs else s1_fn(bits)
        x = mp.fabs(x)
        return STAGE2_TRANSFORMS[best_spec.stage2_name](x)
    base_val = value_from_bits(base_bits)
    return base_val, [value_from_bits(flip_fraction(base_bits, perturb_frac, rng)) for _ in range(trials)]

# ------------------------------------ Run & report ----------------------------------------
def main():
    print("Running fixed-grammar pipelines...")
    results = evaluate_against_targets()
    print(f"Total pipelines (T): {len(PIPELINES)}  => log2(T) = {LOG2_T:.2f} bits")
    print(f"Digits: M_TRAIN={M_TRAIN}, M_TEST={M_TEST} (of total {M_TOTAL})\n")

    print("Computing random baselines...")
    baseline = evaluate_random_baseline(num_runs=20, seed=42)

    rows = []
    for tname, info in results.items():
        if info is None: continue
        spec = info["spec"]; D_train = info["D_train"]; D_test = info["D_test"]
        tval = targets[tname]
        base = baseline.get(tname, None)
        p = sum(1 for d in base["hist"] if d >= D_train)/len(base["hist"]) if base and base["hist"] else float('nan')
        rows.append({
            "target": tname,
            "match_digits_train": D_train,
            "match_digits_test": D_test,
            "surplus_bits_train": info["surplus_bits_train"],
            "rand_mean_max_digits": base["mean"] if base else float('nan'),
            "rand_max_of_max_digits": base["max"] if base else float('nan'),
            "p_value_vs_random": p,
            "pipeline": f"{spec.seq_name} | N={spec.N} | {spec.stage1_name} -> {spec.stage2_name}",
            "approx_value": mp.nstr(info["value"], 20),
            "target_value": mp.nstr(tval, 20),
        })

    rows.sort(key=lambda r: (r["surplus_bits_train"], r["match_digits_train"]), reverse=True)

    print("\n=== Top results by surplus bits (train) ===")
    for r in rows[:12]:
        print(f"- {r['target']:>20s}: D_train={r['match_digits_train']:2d}  D_test={r['match_digits_test']:2d}  "
              f"Surplus={r['surplus_bits_train']:6.2f} bits  p≈{r['p_value_vs_random']:.3f}")
        print(f"    pipeline: {r['pipeline']}")
        print(f"    approx  : {r['approx_value']}")
        print(f"    target  : {r['target_value']}\n")

    if rows:
        best_row = rows[0]; tname = best_row["target"]; spec = results[tname]["spec"]; tval = targets[tname]
        print("Robustness check (flip 1% bits, 20 trials) for best result:\n ", best_row["pipeline"])
        base_val, pert_vals = robustness_check(spec, perturb_frac=0.01, trials=20, seed=7)
        base_D, _ = digits_match(base_val, tval, M_TRAIN, offset=0)
        pert_Ds = [digits_match(v, tval, M_TRAIN, offset=0)[0] for v in pert_vals]
        print(f"    Base D_train={base_D}, after flips mean D_train={statistics.mean(pert_Ds):.2f}, "
              f"min={min(pert_Ds)}, max={max(pert_Ds)}")

    print("\nNote: Surplus ≈ 3.322*D_train - log2(T). K=0 (no fitted continuous parameters); "
          "all discrete choices are predeclared and charged to T.\n")

if __name__ == "__main__":
    main()


Running fixed-grammar pipelines...
Total pipelines (T): 150  => log2(T) = 7.23 bits
Digits: M_TRAIN=14, M_TEST=14 (of total 50)

Computing random baselines...

=== Top results by surplus bits (train) ===
-                  phi: D_train= 3  D_test= 0  Surplus=  2.74 bits  p≈0.150
    pipeline: ThueMorse | N=512 | DirichletPrime_s1 -> reciprocal
    approx  : 1.6192944527513387738
    target  : 1.6180339887498948482

-                sqrt2: D_train= 2  D_test= 0  Surplus= -0.58 bits  p≈1.000
    pipeline: ThueMorse | N=256 | DirichletAll_s2 -> reciprocal
    approx  : 1.442681987366570746
    target  : 1.4142135623730950488

-                   pi: D_train= 1  D_test= 0  Surplus= -3.91 bits  p≈0.900
    pipeline: FibonacciWord | N=256 | BinaryFraction -> reciprocal
    approx  : 3.445940261524254254
    target  : 3.1415926535897932385

-                    e: D_train= 1  D_test= 0  Surplus= -3.91 bits  p≈1.000
    pipeline: FibonacciWord | N=512 | DirichletAll_s1 -> id
    approx  : 2.02